In [1]:
from typing import List
import pandas as pd
import cvxpy as cp
import numpy as np
import torch
import itertools
import scipy

/Users/scortesg/opt/anaconda3/envs/rp_fall2022/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset_size = 100

In [13]:
def simulate_multiple_outcomes(dataset_size: int, _feature_number:int = 4):
    """_summary_

    Args:
        dataset_size (int): _description_

    Returns:
        _type_: _description_
    """
    X = np.random.multivariate_normal(mean=np.zeros(_feature_number), cov=np.identity(_feature_number), size = dataset_size)
    X = pd.DataFrame([pd.cut(X[:,0], [-np.inf, 0, np.inf]).codes,
                    pd.cut(X[:,1], [-np.inf, 1, np.inf]).codes,
                    pd.cut(X[:,2], [-np.inf, -1, np.inf]).codes,
                    pd.cut(X[:,3], [-np.inf, -1, 1, np.inf]).codes]).T
    # TODO (Mateo) Can we simulate with an arbitrary number of features?
    pi_x = scipy.special.expit((2*X[0] - 4*X[1] + 2*X[2] - X[3])/4)
    A = 1*(pi_x > np.random.uniform(size=dataset_size))

    mu_0 = scipy.special.expit(4*A)
    y_0 = 1*(mu_0 > np.random.uniform(size=dataset_size))

    mu_1 = scipy.special.expit(X[0] + X[2] + X[3] + 3*A)
    y_1 = 1*(mu_1 > np.random.uniform(size=dataset_size))

    return X, A, y_0, y_1

In [14]:
X, A, y_0, y_1 = simulate_multiple_outcomes(dataset_size = 100)

In [15]:
X.shape

(100, 4)

#### Data creation problem formulation

A dummie dataset is going to be created, from now on features is going to be a column of the dataset. Levels will be the semantic feature i.e sex and estrata the posible value for a level, i.e male or female

In [16]:
data = pd.read_csv("german_credit_data.csv", index_col=0)
data_labels = pd.read_csv("german_credit.csv")
data_dummies = pd.get_dummies(data["Sex"])

In [17]:
data_dummies[["free", "own", "rent"]] = pd.get_dummies(data["Housing"])
data_dummies[["little", "moderate", "quite rich", "rich"]] = pd.get_dummies(data["Saving accounts"])
data_dummies[["Age Bucket 1", "Age Bucket 2"]] = pd.get_dummies(pd.cut(data.Age, [18, 45, 75]))
data_dummies["Creditability"] = data_labels["Creditability"]

In [18]:
data_dummies.columns

Index(['female', 'male', 'free', 'own', 'rent', 'little', 'moderate',
       'quite rich', 'rich', 'Age Bucket 1', 'Age Bucket 2', 'Creditability'],
      dtype='object')

Now the dataset is artificially biased, in this particular case only one forth of the males with good credit score are taken and one forth of the females with a ba
d one.

In [19]:
income_skewed_data = []
confounders = ["little", "moderate", "quite rich", "rich"]
# sample_prop = np.random.uniform(size=4)
sample_prop = [0.8, 0.8, 0.4, 0.6]

for ii in range(4):
    income_skewed_data.append(data_dummies[data_dummies[confounders[ii]] == 1].sample(frac=sample_prop[ii]))

income_skewed_data = pd.concat(income_skewed_data)

In [20]:
income_skewed_data

,female,male,free,own,rent,little,moderate,quite rich,rich,Age Bucket 1,Age Bucket 2,Creditability
644,0,1,0,1,0,1,0,0,0,1,0,1
981,0,1,0,0,1,1,0,0,0,1,0,0
491,1,0,1,0,0,1,0,0,0,1,0,1
389,1,0,0,1,0,1,0,0,0,1,0,1
365,0,1,0,1,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
167,1,0,0,1,0,0,0,0,1,1,0,1
140,0,1,0,1,0,0,0,0,1,1,0,1
183,0,1,0,1,0,0,0,0,1,0,1,1
733,1,0,0,0,1,0,0,0,1,1,0,1


In [21]:
# females = data_dummies[data_dummies.female == 1]
# females_good = females[females.Creditability == 1]
# females_bad = females[females.Creditability == 0]

# males = data_dummies[data_dummies.female == 0]
# males_good = males[males.Creditability == 1]
# males_bad = males[males.Creditability == 0]

# ma_good_sample = males_good.sample(frac = 0.4)
# fem_bad_sample = females_bad.sample(frac = 0.2)

# ma_bad_sample = males_bad.sample(frac = 0.3)
# females_good_sample = females_good.sample(frac = 0.35)

from now on the random variable $A$ will denote apperaing or not in this datatset.

In [22]:
# sex_skewed_data = pd.concat([ma_good_sample, fem_bad_sample])

### Implementation of `Pending cool name`

In [23]:
def build_counts(data : pd.DataFrame, levels: List[List], target:str):
    """
    Given a dummie variable dataset it return a matrix counts
    from all posible combinations of features
    """
    shape = [0 for i in range(len(levels) + 1)]        
    # 2 comes from the response variable Y in this case binary
    shape[0] = 2
    
    for index, level in enumerate(levels):
        number_levels = len(level)
        shape[index + 1] = number_levels
        
    count = torch.zeros(shape)
    for _, row in data.iterrows():
        position = [0 for i in range(len(levels) + 1)]
        position[0] = row[target]
        
        for index, level in enumerate(levels):
            for index_j, feature in enumerate(level):
                if row[feature] == 1:
                   position[index + 1] = index_j
        count[tuple(position)] += 1
    # Machetimbis para que después no me estallen los gradientes
    count[count == 0] = 0.00001
    return count        

# Watch out
Here is where most of the meat is. The method is based in building restrictions for an optimization problem that will make the observed data consistent with some ground truth value (i.e some data taken from the real world). 

In [24]:
print(data_dummies[data_dummies["Creditability"] == 0][data_dummies["female"] == 1].shape)
print(data_dummies[data_dummies["Creditability"] == 0][data_dummies["male"] == 1].shape)
print(data_dummies[data_dummies["Creditability"] == 1][data_dummies["female"] == 1].shape)
print(data_dummies[data_dummies["Creditability"] == 1][data_dummies["male"] == 1].shape)

(91, 12)
(209, 12)
(219, 12)
(481, 12)


/var/folders/pp/kkyvc6j528n8bbnjl0xkn20r0000gp/T/ipykernel_9234/1081995785.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(data_dummies[data_dummies["Creditability"] == 0][data_dummies["female"] == 1].shape)
/var/folders/pp/kkyvc6j528n8bbnjl0xkn20r0000gp/T/ipykernel_9234/1081995785.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(data_dummies[data_dummies["Creditability"] == 0][data_dummies["male"] == 1].shape)
/var/folders/pp/kkyvc6j528n8bbnjl0xkn20r0000gp/T/ipykernel_9234/1081995785.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(data_dummies[data_dummies["Creditability"] == 1][data_dummies["female"] == 1].shape)
/var/folders/pp/kkyvc6j528n8bbnjl0xkn20r0000gp/T/ipykernel_9234/1081995785.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(data_dummies[data_dummies["Creditability"] == 1][data_dummies["male"] == 1].shape)


In [25]:
# Hand made ground truth restricitions
b0 = np.array([91, 209])
b1 = np.array([219, 481])

In [26]:
def build_strata_counts_matrix(weight_features: torch.Tensor, counts: torch.Tensor, level: List[str]):
    """Builds linear restrictions for a convex opt problem.
    
    This method build a Matrix with counts by combination of strata,
    It will return two matrixes each one associated to the idividuals
    with y=1 or y=1:
    
    A_1 (a_{ij}), a_{ij} is the number of observations in the dataset
    such that y=1 x_i = 1 and x_j =1.
    
    A_0 (a_{ij}), a_{ij} is the number of observations in the dataset
    such that y=1 x_i = 1 and x_j =1.
    
    Note that unlike i, j is fixed, in the code outside this method
    varies trough female and male
    
    returns A_0, A_1
    """
    _, features = weight_features.shape
    level_size = len(level)
    # This should be associated with the y?
    
    y_0_ground_truth = torch.zeros(level_size, features)
    y_1_ground_truth = torch.zeros(level_size, features)
    
    data_count_0 = counts[0]
    data_count_1 = counts[1]
    
    for level in range(level_size):
        # Flaw here only works with the first level.
        t = data_count_0[level].flatten().unsqueeze(1)
        features = weight_features[level*t.shape[0]:(level + 1)*t.shape[0]]        
        y_0_ground_truth[level] = (features*t).sum(dim=0)

        
    for level in range(level_size):
        # Flaw here only works with the first level.
        t_ = data_count_1[level].flatten().unsqueeze(1)    
        features_ = weight_features[weight_features.shape[0]//2 + level*t.shape[0]:weight_features.shape[0]//2 + (level + 1)*t.shape[0]]        
        y_1_ground_truth[level] = (features_*t_).sum(dim=0)
        

    return y_0_ground_truth, y_1_ground_truth
        

This method was built for ease of computation, we wanted something that can be written in terms of matrix multiplications and slices thus taking advantage of DL frameworks paralelization.

In [27]:
# Put always the level we want to aggregate from before than anything
# levels = [["female", "male"], ["Age Bucket 1", "Age Bucket 2"]]
levels = [['female', 'male'], ['free', 'own', 'rent'], ['little', 'moderate',
       'quite rich', 'rich'], ['Age Bucket 1', 'Age Bucket 2']]

In [28]:
counts = build_counts(income_skewed_data, levels, "Creditability")

In [40]:
data_count_0 = counts[0]
data_count_1 = counts[1]

In [41]:
weights_features = torch.zeros(counts.numel(), 9)
idx = 0
for target in [0, 1]:
    for i, sex in enumerate(["female", "male"]):
        for housing in ['free', 'own', 'rent']:
            for j, income in enumerate(['little', 'moderate', 'quite rich', 'rich']):
                for age in ['Age Bucket 1', 'Age Bucket 2']:
                    credit_sex_features = [0]*4
                    credit_sex_features[target*2 + i] = 1
                    income_features = [0]*4
                    income_features[j] = 1
                    weights_features[idx] = torch.tensor(credit_sex_features + income_features + [1]).float()
                    idx += 1

In [42]:
A0, A1 = build_strata_counts_matrix(weights_features, counts, ["female", "male"])

In [43]:
A0[0]

tensor([57.0001,  0.0000,  0.0000,  0.0000, 44.0000,  7.0000,  4.0000,  2.0001,
        57.0001])

In [44]:
A1[0]

tensor([  0.0000,   0.0000, 141.0001,   0.0000, 108.0000,  21.0000,   2.0000,
         10.0000, 141.0001])

In [45]:
weights_features.shape

torch.Size([96, 9])

### Optimization part
Let it rip!!

In [46]:
aux = income_skewed_data.groupby('female')['Creditability'].mean()
aux[1] - aux[0]

0.019264069264069317

In [47]:
aux = data_dummies.groupby('female')['Creditability'].mean()
gt_ate= aux[1] - aux[0]
gt_ate

0.00935016362786345

In [50]:
torch.autograd.set_detect_anomaly(True)
alpha = torch.rand(weights_features.shape[1], requires_grad=True)
W = np.unique(weights_features.numpy(), axis=0)
tol = 0.00000001

optim = torch.optim.Adam([alpha], lr=5e-4)

for iteration in range(10000):
    w = cp.Variable(alpha.shape[0])
    alpha_fixed = alpha.squeeze().detach().numpy()
    A_0 = A0.numpy()
    A_1 = A1.numpy()

    objective = cp.sum_squares(w - alpha_fixed)
    # With the two rstrictions (as it should) the problem is infeasible W >=1 ????
    restrictions = [A_0@ w == b0, A_1@ w == b1]
    prob = cp.Problem(cp.Minimize(objective), restrictions)
    prob.solve()

    alpha.data = torch.tensor(w.value).float()
    weights_y1 = (weights_features[weights_features.shape[0]//2:]@alpha).reshape(*data_count_0.shape)
    weights_y0 = (weights_features[:weights_features.shape[0]//2]@alpha).reshape(*data_count_1.shape)
    
    
    weighted_counts_1 = weights_y1*data_count_1
    weighted_counts_0 = weights_y0*data_count_0

    sex = 1
    sex_base = 0
    
    probs = weighted_counts_1/(weighted_counts_1 + weighted_counts_0)

    
    total_weight_count = weighted_counts_1[sex] + weighted_counts_0[sex]
    ATE = ((probs[sex] - probs[sex_base])*total_weight_count/total_weight_count.sum()).sum()
    
    loss = ATE
    if iteration % 500 == 0:
        print(ATE.item(), ATE.item()/gt_ate, gt_ate)
    
    optim.zero_grad()
    loss.backward()
    optim.step()
    

-0.027288595214486122 -2.9185152581893026 0.00935016362786345


KeyboardInterrupt: 

In [39]:
0.009537313133478165, -0.034625258296728134

(0.009537313133478165, -0.034625258296728134)

In [37]:
torch.autograd.set_detect_anomaly(True)
alpha = torch.rand(weights_features.shape[1], requires_grad=True)
W = np.unique(weights_features.numpy(), axis=0)
tol = 0.00000001

optim = torch.optim.Adam([alpha], lr=5e-2)

for iteration in range(10000):
    w = cp.Variable(alpha.shape[0])
    alpha_fixed = alpha.squeeze().detach().numpy()
    A_0 = A0.numpy()
    A_1 = A1.numpy()

    objective = cp.sum_squares(w - alpha_fixed)
    # With the two rstrictions (as it should) the problem is infeasible W >=1 ????
    restrictions = [A_0@ w == b0, A_1@ w == b1, w>=1]
    prob = cp.Problem(cp.Minimize(objective), restrictions)
    prob.solve()

    #
    alpha.data = torch.tensor(w.value).float()
    weights_y1 = (weights_features[weights_features.shape[0]//2:]@alpha).reshape(*data_count_0.shape)
    weights_y0 = (weights_features[:weights_features.shape[0]//2]@alpha).reshape(*data_count_1.shape)
    
    
    weighted_counts_1 = weights_y1*data_count_1
    weighted_counts_0 = weights_y0*data_count_0

    sex = 1
    sex_base = 0
    
    
    probs = weighted_counts_1/(weighted_counts_1 + weighted_counts_0)

    
    total_weight_count = weighted_counts_1[sex] + weighted_counts_0[sex]
    # Here is the g-formula, The assertation now is sex
    
    diff = ((probs[sex] - probs[sex_base])*total_weight_count/total_weight_count.sum()).sum()
    total_weighted_count_base =  weighted_counts_1[sex_base] + weighted_counts_0[sex_base]
    base = ((probs_gt[sex] - probs_gt[sex_base])*total_weight_count_gt/total_weight_count_gt.sum()).sum()
    
    loss = diff
    if iteration % 500 == 0:
        print(diff.item(), diff.item()/base.item(), base.item())
    
    optim.zero_grad()
    loss.backward()
    optim.step()
    

RuntimeError: Could not infer dtype of NoneType

In [16]:
# Put always the level we want to aggregate from before than anything
# levels = [["female", "male"], ["Age Bucket 1", "Age Bucket 2"]]
levels = [['female', 'male'], ['free', 'own', 'rent'], ['little', 'moderate',
       'quite rich', 'rich'], ['Age Bucket 1', 'Age Bucket 2']]

In [90]:
torch.autograd.set_detect_anomaly(True)
alpha = torch.rand(weights_features.shape[1], requires_grad=True)
W = np.unique(weights_features.numpy(), axis=0)
tol = 0.00000001

optim = torch.optim.Adam([alpha], lr=5e-2)

for iteration in range(10000):
    w = cp.Variable(alpha.shape[0])
    alpha_fixed = alpha.squeeze().detach().numpy()
    A_0 = A0.numpy()
    A_1 = A1.numpy()

    objective = cp.sum_squares(w - alpha_fixed)
    # With the two rstrictions (as it should) the problem is infeasible W >=1 ????
    restrictions = [A_0@ w == b0, A_1@ w == b1, w>=1]
    prob = cp.Problem(cp.Minimize(objective), restrictions)
    prob.solve()


    alpha.data = torch.tensor(w.value).float()
    weights_y1 = (weights_features[weights_features.shape[0]//2:]@alpha).reshape(*data_count_0.shape)
    weights_y0 = (weights_features[:weights_features.shape[0]//2]@alpha).reshape(*data_count_1.shape)
    

    weighted_counts_1 = weights_y1*data_count_1
    weighted_counts_0 = weights_y0*data_count_0


    conditioned_counts_0 = torch.select(data_count_0, 1, 1)
    conditioned_counts_1 = torch.select(data_count_1, 1, 1)


    weighted_conditioned_counts_0 = torch.select(weighted_counts_0, 1, 1)
    weighted_conditioned_counts_1 = torch.select(weighted_counts_1, 1, 1)
    
    base = (conditioned_counts_1.sum()/(conditioned_counts_1 + conditioned_counts_0).sum())
    prob = weighted_conditioned_counts_1.sum()/(weighted_conditioned_counts_0 + weighted_conditioned_counts_1).sum()

    # Predict probability by income
    loss = prob
    if iteration % 500 == 0:
        print(prob.item() , prob.item()/base.item(), base.item())
    
    optim.zero_grad()
    loss.backward()
    optim.step()
    

0.7697690725326538 0.8259571660035132 0.9319721460342407
0.7751370668411255 0.8317169886884654 0.9319721460342407
0.7751371264457703 0.8317170526438584 0.9319721460342407
0.7751370668411255 0.8317169886884654 0.9319721460342407
0.7751370668411255 0.8317169886884654 0.9319721460342407
0.7751370668411255 0.8317169886884654 0.9319721460342407
0.7751370668411255 0.8317169886884654 0.9319721460342407
0.7751370668411255 0.8317169886884654 0.9319721460342407
0.7751370668411255 0.8317169886884654 0.9319721460342407
0.7751370668411255 0.8317169886884654 0.9319721460342407
0.7751370668411255 0.8317169886884654 0.9319721460342407
0.7752014994621277 0.8317861244681949 0.9319721460342407
0.7752014398574829 0.831786060512802 0.9319721460342407


KeyboardInterrupt: 

In [ ]:
levels = [['female', 'male'], ['free', 'own', 'rent'], ['little', 'moderate',
       'quite rich', 'rich'], ['Age Bucket 1', 'Age Bucket 2']]

In [22]:
data_dummies[data_dummies["own"] == 1].sum()/data_dummies.shape[0]

female           0.196
male             0.517
free             0.000
own              0.713
rent             0.000
little           0.430
moderate         0.071
quite rich       0.045
rich             0.037
Age Bucket 1     0.590
Age Bucket 2     0.123
Creditability    0.504
dtype: float64

In [23]:
sex_skewed_data[sex_skewed_data["Age Bucket 1"] == 1].sum()/sex_skewed_data.shape[0]

female           0.047619
male             0.652381
free             0.000000
own              0.700000
rent             0.000000
little           0.404762
moderate         0.076190
quite rich       0.057143
rich             0.028571
Age Bucket 1     0.566667
Age Bucket 2     0.133333
Creditability    0.652381
dtype: float64

In [40]:
print((data_count_1 + data_count_0).sum(), sex_skewed_data.shape[0])

tensor(210.0007) 210


In [51]:
sex_skewed_data.sum()

female            18
male             192
free              30
own              147
rent              33
little           120
moderate          22
quite rich        18
rich               6
Age Bucket 1     164
Age Bucket 2      46
Creditability    192
dtype: int64

In [91]:
sex_skewed_data.sum()/sex_skewed_data.shape[0]

female           0.085714
male             0.914286
free             0.142857
own              0.700000
rent             0.157143
little           0.571429
moderate         0.104762
quite rich       0.085714
rich             0.028571
Age Bucket 1     0.780952
Age Bucket 2     0.219048
Creditability    0.914286
dtype: float64

In [95]:
sex_skewed_data[sex_skewed_data["little"] == 1].sum()/sex_skewed_data[sex_skewed_data["little"] == 1].shape[0]

female           0.075000
male             0.925000
free             0.141667
own              0.708333
rent             0.150000
little           1.000000
moderate         0.000000
quite rich       0.000000
rich             0.000000
Age Bucket 1     0.808333
Age Bucket 2     0.191667
Creditability    0.925000
dtype: float64

In [76]:
sex_skewed_data[sex_skewed_data["own"] == 1].sum()

female            10
male             137
free               0
own              147
rent               0
little            85
moderate          16
quite rich        12
rich               6
Age Bucket 1     119
Age Bucket 2      28
Creditability    137
dtype: int64

In [75]:
data_count_1.sum()/(data_count_1 + data_count_0).sum()

tensor(0.9143)

In [77]:
torch.select(torch.select(counts, 2, 1), 0, 1).sum()/torch.select(counts, 2, 1).sum()

tensor(0.9320)

In [44]:
data_count_1 + data_count_0

tensor([[[[2.0000e-05, 1.0000e+00],
          [2.0000e-05, 2.0000e-05],
          [2.0000e-05, 2.0000e-05],
          [2.0000e-05, 2.0000e-05]],

         [[7.0000e+00, 1.0000e+00],
          [2.0000e-05, 2.0000e-05],
          [2.0000e-05, 1.0000e+00],
          [1.0000e+00, 2.0000e-05]],

         [[5.0000e+00, 2.0000e-05],
          [1.0000e+00, 2.0000e-05],
          [1.0000e+00, 2.0000e-05],
          [2.0000e-05, 2.0000e-05]]],


        [[[1.3000e+01, 1.3000e+01],
          [2.0000e+00, 2.0000e-05],
          [2.0000e-05, 1.0000e+00],
          [2.0000e-05, 2.0000e-05]],

         [[8.5000e+01, 2.0000e+01],
          [1.5000e+01, 1.0000e+00],
          [8.0000e+00, 3.0000e+00],
          [3.0000e+00, 2.0000e+00]],

         [[1.8000e+01, 1.0000e+00],
          [2.0000e+00, 1.0000e+00],
          [3.0000e+00, 1.0000e+00],
          [2.0000e-05, 2.0000e-05]]]])

In [42]:
(data_count_1.sum() / (data_count_1 + data_count_0).sum())

tensor(0.9143)

In [38]:
data_count_1

tensor([[[[1.0000e-05, 1.0000e-05],
          [1.0000e-05, 1.0000e-05],
          [1.0000e-05, 1.0000e-05],
          [1.0000e-05, 1.0000e-05]],

         [[1.0000e-05, 1.0000e-05],
          [1.0000e-05, 1.0000e-05],
          [1.0000e-05, 1.0000e-05],
          [1.0000e-05, 1.0000e-05]],

         [[1.0000e-05, 1.0000e-05],
          [1.0000e-05, 1.0000e-05],
          [1.0000e-05, 1.0000e-05],
          [1.0000e-05, 1.0000e-05]]],


        [[[1.3000e+01, 1.3000e+01],
          [2.0000e+00, 1.0000e-05],
          [1.0000e-05, 1.0000e+00],
          [1.0000e-05, 1.0000e-05]],

         [[8.5000e+01, 2.0000e+01],
          [1.5000e+01, 1.0000e+00],
          [8.0000e+00, 3.0000e+00],
          [3.0000e+00, 2.0000e+00]],

         [[1.8000e+01, 1.0000e+00],
          [2.0000e+00, 1.0000e+00],
          [3.0000e+00, 1.0000e+00],
          [1.0000e-05, 1.0000e-05]]]])

In [35]:
data_count_1_s = torch.select(data_count_1, 1, 1)
data_count_0_s = torch.select(data_count_0, 1, 1)

In [37]:
data_count_1_s/(data_count_1_s + data_count_0_s).sum()

tensor([[[6.8027e-08, 6.8027e-08],
         [6.8027e-08, 6.8027e-08],
         [6.8027e-08, 6.8027e-08],
         [6.8027e-08, 6.8027e-08]],

        [[5.7823e-01, 1.3605e-01],
         [1.0204e-01, 6.8027e-03],
         [5.4422e-02, 2.0408e-02],
         [2.0408e-02, 1.3605e-02]]])

In [32]:
4.7619e-08 

4.7619e-08

In [33]:
0.047619 == 4.7619e-08 

False